# Notes

This notebook creates subhalo ID and particle ID lists for snapshots for IllustrisTNG simulations.

These can be loaded and used for cross-matching for impact factors to galaxies.

The code is updated from: 

raven:/u/cwalker/Illustris_Zhang_Method/Create_Particle_Subhalo_Matchlists_4.ipynb

using:

raven:/u/cwalker/Illustris_Zhang_Method/test_artale_vs_yt_field_load.ipynb

- See previous versions for detailed notes about development.
- It updates locations of TNG data after /virgo/ disk errors.
- This version fixes issue where chunks created by the previous version for the TNG300-1 matchlists were different sizes. As discovered by WHIM_Pipes_Subhalo_Analysis.ipynb, the particle ID chunks were one element shorter than the subhalo ID chunks. This issue arose in process_matchlist_chunk() in Create_Particle_Subhalo_Matchlists_4.ipynb.

# Imports

In [6]:
import os
import yt
import h5py
import random

import numpy as np
import multiprocessing as m
import illustris_python as il

from contextlib import closing
from multiprocessing import Pool

from matplotlib import pyplot as plt
from charlie_TNG_tools import pID2shID


# Functions

In [7]:
def unwrap_matchlist_package(package):
    """
    Helper function for making simulation matchlists in parallel using multiprocessing.
    Unpacks the set of data necessary for parsing the simulation and making matchlists
    for certain chunks. Then makes those chunks with process_matchlist_chunk().
    
    
    INPUTS:
    
    package : a list containing the inpyt data, which are X arguments in the following order:
    
        nchunks_total   : [int] the total number of chunks the simulation will be split into to create
                        matchlist covering the entire simulation
        chunkIDs        : [array of ints] the id numbers of the chunks of simulation to be processed
                        on this cpu
        indices         : [array] array of particle indices to load for each whole chunk. shape of 
                        (nchunks_total,2)
        simPath         : [str] location of sim file to be loaded as hdf5 File containing all the particle IDs
                        subhalo IDs will be created for
        partIDs_loc     : [str] location of the (partType0) particle IDs for this simulation snapshot
        basePath        : [str] base path to the simulation data
        snapshot_number : [int] the snapshot of the simulation to process
        offsetFile      : [str] location of the offset file for this simulation and snapshot
        subdir_name     : [str] location of subdirectory to store output matchlist chunk in
    
    RETURNS:
    
    output of process_matchlist_chunk()
    
    """
    
    verbose=True
    
    #unwrap the package to feed to process_matchlist_chunk()
    nchunks_total   = package[0]
    if verbose==True:
        print('Verbose mode check: nchunks_total = {0}'.format(nchunks_total))
    chunkIDs        = package[1]
    if verbose==True:
        print('Verbose mode check: chunkIDs = {0}'.format(chunkIDs))
    indices         = package[2]
    if verbose==True:
        print('Verbose mode check: indices = {0}'.format(indices))
    simPath         = package[3]
    if verbose==True:
        print('Verbose mode check: simPath = {0}'.format(simPath))
    partIDs_loc     = package[4]
    if verbose==True:
        print('Verbose mode check: partIDs_loc = {0}'.format(partIDs_loc))
    basePath        = package[5]
    if verbose==True:
        print('Verbose mode check: basePath = {0}'.format(basePath))
    snapshot_number = package[6]
    if verbose==True:
        print('Verbose mode check: snapshot_number = {0}'.format(snapshot_number))
    offsetFile      = package[7]
    if verbose==True:
        print('Verbose mode check: offsetFile = {0}'.format(offsetFile))
    subdir_name     = package[8]
    if verbose==True:
        print('Verbose mode check: subdir_name = {0}'.format(subdir_name))
    
    #run process_matchlist_chunk()
    process_matchlist_chunk(nchunks_total,chunkIDs,indices,simPath,partIDs_loc,basePath,snapshot_number,offsetFile,subdir_name)
    
    return 'done'

def process_matchlist_chunk(nchunks_total,
                            chunkIDs,
                            indices,
                            simPath,
                            partIDs_loc,
                            basePath,
                            snapshot_number,
                            offsetFile,
                            subdir_name):
    """

    For a simulation snapshot split into nchunks chunks,  creates certain chunks of the simulation 
    particle/subhalo ID matchlist on a single cpu. It is fed by unwrap_matchlist_package().
    
    
    INPUTS:
    
    
    nchunks_total   : [int] the total number of chunks the simulation will be split into to create
                    matchlist covering the entire simulation
    chunkIDs        : [array of ints] the id numbers of the chunks of simulation to be processed
                    on this cpu
    indices         : [array] array of particle indices to load for each whole chunk. shape of 
                    (nchunks_total,2)
    simPath         : [str] location of sim file to be loaded as hdf5 File containing all the particle IDs  
                    subhalo IDs will be created for
    partIDs_loc     : [str] location of the (partType0) particle IDs for this simulation snapshot
    basePath        : [str] base path to the simulation data
    snapshot_number : [int] the snapshot of the simulation to process
    offsetFile      : [str] location of the offset file for this simulation and snapshot
    subdir_name     : [str] location of subdirectory to store output matchlist chunk in
    
    RETURNS
    
    1 (must return something so multiprocessing is guaranteed to terminate after process is finished)
    
    """
    
    #print(simPath)

    #get all gas particle IDs in the simulation snapshot
    with h5py.File(simPath) as f:

        print(f)
        
        #for the chunks of the simulation to be processed by this CPU...
        for i in range(len(chunkIDs)):
            
            ########################################
            #CREATE THE CHUNK PARTICLE ID MATCHLIST#
            ########################################
            
            #select the chunk ID to be processed
            chunk_id = chunkIDs[i] 

            #extract the indices within this chunk which will be loaded
            print('loading chunk {0}'.format(chunk_id))
            chunk_first_idx = indices[chunk_id][0]
            chunk_last_idx = indices[chunk_id][-1]
            print('chunk indices: ({0},{1})'.format(chunk_first_idx,chunk_last_idx))

            #load only the particles in this chunk, according to the indices
            #allparts = np.copy(f[partIDs_loc][chunk_first_idx:chunk_last_idx])
            #edit 04/07/2022 to make particle ID chunks match subhalo ID chunks.
            allparts = np.copy(f[partIDs_loc][chunk_first_idx:chunk_last_idx+1])
            
            #get number of particles in this chunk
            nparts = allparts.shape[0] 
            print('chunk loaded.')

            #create a list version of every particle ID in this chunk
            AllPartList = allparts.tolist()
            print('Particle list created')

            #######################################
            #CREATE THE CHUNK SUBHALO ID MATCHLIST#
            #######################################

            #choose which particles to ID
            to_ID = np.arange(chunk_first_idx,chunk_last_idx+1) #replace with the indices we want to load    

            #choose the particle type (note: 0 is gas, and it should always be gas, but perhaps this)
            #should not be hard coded...
            partType = 0 #gas

            #choose subhalo fields to load
            subhaloFields = ['SubhaloFlag',
                             'SubhaloPos',
                             'SubhaloHalfmassRad',
                             'SubhaloHalfmassRadType',
                             'SubhaloLenType']

            #load subhalo catalog
            subhalos=il.groupcat.loadSubhalos(basePath,snapshot_number,fields=subhaloFields) 
            subhalos.keys()

            #get subhalo offset file for matching particle and subhalo IDs
            with h5py.File(offsetFile,'r') as f2:
                SnapOffsetsSubhalo= np.copy(f2['/Subhalo/SnapByType'])
            print('offsetfile loaded')

            #get subhalo lengths for all gas particles
            SubhaloLenType = np.copy(subhalos['SubhaloLenType'])

            #create array of subhaloIDs for every gas particle
            AllShIDList = pID2shID(to_ID,partType,SubhaloLenType,SnapOffsetsSubhalo)
            print('ShIDList created')

            #NOTE: you can run this for TNG300-1 in parts and save them in parts. you would then have to amend the loading
            #in the pipe creation code to parse through each part separately.
            #uniquestest_chunked.append(np.unique(AllShIDList).tolist())
            
            ############################
            #SAVE BOTH MATCHLIST CHUNKS#
            ############################
            
            np.save('/{0}/PartList_Snap{1}_Chunk{2:04d}.npy'.format(subdir_name,snapshot_number,chunk_id),AllPartList)
            np.save('/{0}/ShIDList_Snap{1}_Chunk{2:04d}.npy'.format(subdir_name,snapshot_number,chunk_id),AllShIDList)  
            print('saved')
    return 1

# Initialise

In [8]:
#simulation to create lists for
sim_to_use = 'TNG300-1'

#snapshots to create lists for
snap_list = [91]#[99,91,84,78,72,67,59,50,40,33,25,21,17]

#base path to data
basePath = '/virgotng/universe/IllustrisTNG/{0}/output/'.format(sim_to_use)

#path to simulation hdf5 file
simPath = '/virgotng/universe/IllustrisTNG/{0}/simulation.hdf5'.format(sim_to_use)

#check these exist
print('Testing whether basePath and simPath exist...')
print('basePath exists = {0}'.format(os.path.exists(basePath)))
print('simPath exists = {0},\n{1}'.format(os.path.exists(simPath),simPath))

nchunks=1000 #the desied number of chunks (minus remainder) to process the particle data in

#identify number of available cores on the system
ncpus = m.cpu_count()
print('Maximum of {0} cores available to use.'.format(ncpus))

cpus_to_use = 8
print('Desired number of cores to use: {0}'.format(cpus_to_use))



Testing whether basePath and simPath exist...
basePath exists = True
simPath exists = True,
/virgotng/universe/IllustrisTNG/TNG300-1/simulation.hdf5
Maximum of 144 cores available to use.
Desired number of cores to use: 8


# Create directories to store matchlists

In [9]:
#check to see if a top directory exists
#topdir_name = '/u/cwalker/Illustris_Zhang_Method/Sim_Matchlists/'
topdir_name = '/ptmp/cwalker/Illustris_FRB_Project/Sim_Matchlists/' #edit 14/03/2022: put in /ptmp/ now due to size for large simulations
topdir_check = os.path.exists(topdir_name)

#if it exists, print
if topdir_check == True:
    print('Top directory ({0}) already exists.'.format(topdir_name))
    
#else, create it.
elif topdir_check == False:
    print('Creating top directory for matchlists at {0}...'.format(topdir_name))
    os.mkdir(topdir_name)
    print('{0} created.'.format(topdir_name))
    
    
#check to see if subdirectory for particular simulation matchlist exists
subdir_name = topdir_name+'Matchlist_dir_{0}'.format(sim_to_use)
subdir_check = os.path.exists(subdir_name)

#if it exists, print
if subdir_check == True:
    print('Directory to hold {0} matchlist ({1} exists.)'.format(sim_to_use,subdir_name))

#else, create it
elif subdir_check == False:
    print('Creating subdirectory {0}...'.format(subdir_name))
    os.mkdir(subdir_name)
    print('{0} created.'.format(subdir_name))

Top directory (/ptmp/cwalker/Illustris_FRB_Project/Sim_Matchlists/) already exists.
Directory to hold TNG300-1 matchlist (/ptmp/cwalker/Illustris_FRB_Project/Sim_Matchlists/Matchlist_dir_TNG300-1 exists.)


# Create matchlists

In [ ]:
#loop over snapshots
for snapshot_number in snap_list:
    
    uniquestest_chunked = [] #test array to record all unique subhalo IDs to test against the non-chunked version
    
    print('Creating for {0} snap {1}'.format(sim_to_use,snapshot_number))
    
     #########################
     #SNAPSHOT DATA LOCATIONS#
     #########################

    offsetFile = '/virgotng/universe/IllustrisTNG/{0}/postprocessing/offsets/offsets_0{1}.hdf5'.format(sim_to_use,snapshot_number)
    data_loc = '/virgotng/universe/IllustrisTNG/{0}/output/snapdir_0{1}/snap_0{1}.0.hdf5'.format(sim_to_use,snapshot_number)
    partIDs_loc = '/Snapshots/{0}/PartType0/ParticleIDs'.format(snapshot_number)
    
    print('Processing snapshot {0} at:\n{1}\n with offset file:\n{2}\n and particle IDs file loc:\n{3}'.format(snapshot_number,data_loc,offsetFile,partIDs_loc))

    
    #########################
    #CREATE PARTICLE ID LIST#
    #########################
    
    
    #get all gas particle IDs in snapshot
    with h5py.File(simPath) as f:
        
        
        #you can load smaller chunks of this at a time! Exploit this.
        
        
        simparts = f[partIDs_loc].shape[0] #get the total number of particles in the simulation
        
        ########################
        ########################
        ##CHUNKING INFORMATION##
        ########################
        ########################
        
        #######################################################################################
        #the number of chunks of data to load the particle IDs in according to desired nchunks#
        #######################################################################################
        
        print('calculating chunking information for {0} particles over {1} desired chunks...'.format(simparts,
                                                                                                     nchunks))
        
        nchunks_whole = nchunks #number of whole chunks to process
        parts_per_chunk = simparts//nchunks #number of particles in a whole chunk which will be processed
        remainder = simparts%nchunks #number of particles in the remainder once whole chunks are processed.
        if remainder!=0: #if there is any remainder...
            nchunks_partial = 1 #one extra chunk will be processed, which is not the same size as the others
        else: #if there is no remainder...
            nchunks_partial = 0 #no extra chunks will be processed.
        nchunks_total = nchunks_whole + nchunks_partial #the total number of chunks (plus any remainder) to process
        
        print('will process simulation in {0} chunks. {1} chunks of {2} particles, and {3} chunks of {4}.'.format(nchunks_total,
                                                                                                                 nchunks_whole,
                                                                                                                  parts_per_chunk,
                                                                                                                  nchunks_partial,
                                                                                                                  remainder))
        ###############################################################################                                                                                                    
        #how this loading will be distributed over desired number of cores cpus_to_use#  
        ###############################################################################

        #choose the number of cores to use at once. 
        cpus_to_use = cpus_to_use 

        #calculate the number of full core runs to be used to check for simulation cells in pipe
        #this number is the number of parts of the simulation which will be loaded simultaneously
        n_full_core = nchunks_total//cpus_to_use
        print(n_full_core)
        

        #calculate the number of cores which must be used to check the remaining simulation cells
        #this number is the number of leftover parts of the simulation which will be loaded all at once
        n_partial_core = nchunks_total%cpus_to_use

        print('To parse these {0} chunks over {1} cpus, {1} cpus will load data simultaneously.\n\
              This will happen {2} times. The remaining data needs {3} cpus.\n\
              These will be loaded simultaneously.'.format(nchunks_total,
                                                           cpus_to_use,
                                                           n_full_core,
                                                           n_partial_core))
        
        ###################################
        #indices of chunks (and remainder)#
        ###################################
        
        #calculate the particle indices to load for each whole chunk
        indices = [[i*parts_per_chunk,(i*parts_per_chunk)+(parts_per_chunk-1)] for i in range(nchunks_whole)]
        #append the particle indices for the remainder chunk
        indices.append([nchunks*(parts_per_chunk),(nchunks*parts_per_chunk)-1+(remainder)])
        indices=np.array(indices)
        print('particles will be loaded according to indices array of shape {0}:\n\
        {1}'.format(indices.shape,
                    indices))
        
    ###################                   
    ###################                                                                                                          
    ##PROCESSING DATA##                                                                                                          
    ###################
    ###################

    if n_partial_core == 0: #if there is no remainder to process after full core runs

        print('No partial cores to process')

        ####################################################
        #map of which indices will be processed on each cpu#
        ####################################################

        cpu_map = np.arange(n_full_core*cpus_to_use).reshape(cpus_to_use,n_full_core) 
        print('CPU map: {0}'.format(cpu_map))

        ############################################################################
        #package of data to be passed to an unwrapper to unwrap for multiprocessing#
        ############################################################################

        package = [(nchunks_total,
                    cpu_map[i],
                    indices,
                    simPath,
                    partIDs_loc,
                    basePath,
                    snapshot_number,
                    offsetFile,
                    subdir_name) for i in range(cpus_to_use)]
        #print('package: {0}'.format(package))

        #########################
        #the multiprocessing bit#
        #########################

        with closing(Pool(cpus_to_use)) as p: #invoke multiproccessing
            run = p.map(unwrap_matchlist_package,package,chunksize=1) #run the multiprocessing
        p.terminate() #terminate after completion
        p.join() #eeven more terminating

    elif n_partial_core > 0: #else if there is remainder to process after full core runs

        print('Partial cores to process')

        #####################################################
        #maps of which indices will be processed on each cpu#
        #####################################################

        cpu_map_a = np.arange(n_full_core*cpus_to_use).reshape(cpus_to_use,n_full_core)
        cpu_map_b = np.arange(n_full_core*cpus_to_use,nchunks_total).reshape(n_partial_core,1)
        print('CPU map a : {0}'.format(cpu_map_a))
        print('CPU map b : {0}'.format(cpu_map_b))
        print(indices[cpu_map_b])

        #############################################################################
        #packages of data to be passed to an unwrapper to unwrap for multiprocessing#
        #############################################################################

        package_a = [(nchunks_total,
                      cpu_map_a[i],
                      indices,
                      simPath,
                      partIDs_loc,
                      basePath,
                      snapshot_number,
                      offsetFile,
                      subdir_name) for i in range(cpus_to_use)]
        package_b = [(nchunks_total,
                      cpu_map_b[i],
                      indices,
                      simPath,
                      partIDs_loc,
                      basePath,
                      snapshot_number,
                      offsetFile,
                      subdir_name) for i in range(n_partial_core)]
        #print('package a: {0}'.format(package_a))
        #print('package b: {0}'.format(package_b))

        #########################
        #the multiprocessing bit#
        #########################

        print('full core')
        with closing(Pool(cpus_to_use)) as p: #invoke multiproccessing
            run = p.map(unwrap_matchlist_package,package_a,chunksize=1) #run the multiprocessing
        p.terminate() #terminate after completion
        p.join() #even more terminating

        print('partial core')
        with closing(Pool(n_partial_core)) as p: #invoke multiproccessing
            run = p.map(unwrap_matchlist_package,package_b,chunksize=1) #run the multiprocessing
        p.terminate() #terminate after completion
        p.join() #even more terminating


Creating for TNG300-1 snap 91
Processing snapshot 91 at:
/virgotng/universe/IllustrisTNG/TNG300-1/output/snapdir_091/snap_091.0.hdf5
 with offset file:
/virgotng/universe/IllustrisTNG/TNG300-1/postprocessing/offsets/offsets_091.hdf5
 and particle IDs file loc:
/Snapshots/91/PartType0/ParticleIDs


/raven/u/cwalker/virtual_environments/yt-env/lib/python3.7/site-packages/ipykernel_launcher.py:25: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.


calculating chunking information for 14492458417 particles over 1000 desired chunks...
will process simulation in 1001 chunks. 1000 chunks of 14492458 particles, and 1 chunks of 417.
125
To parse these 1001 chunks over 8 cpus, 8 cpus will load data simultaneously.
              This will happen 125 times. The remaining data needs 1 cpus.
              These will be loaded simultaneously.
particles will be loaded according to indices array of shape (1001, 2):
        [[          0    14492457]
 [   14492458    28984915]
 [   28984916    43477373]
 ...
 [14463473084 14477965541]
 [14477965542 14492457999]
 [14492458000 14492458416]]
Partial cores to process
CPU map a : [[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
   18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
   36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
   54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
   72  73  74  75  76  77  7

 733 734 735 736 737 738 739 740 741 742 743 744 745 746 747 748 749]
Verbose mode check: chunkIDs = [500 501 502 503 504 505 506 507 508 509 510 511 512 513 514 515 516 517
 518 519 520 521 522 523 524 525 526 527 528 529 530 531 532 533 534 535
 536 537 538 539 540 541 542 543 544 545 546 547 548 549 550 551 552 553
 554 555 556 557 558 559 560 561 562 563 564 565 566 567 568 569 570 571
 572 573 574 575 576 577 578 579 580 581 582 583 584 585 586 587 588 589
 590 591 592 593 594 595 596 597 598 599 600 601 602 603 604 605 606 607
 608 609 610 611 612 613 614 615 616 617 618 619 620 621 622 623 624]
Verbose mode check: chunkIDs = [875 876 877 878 879 880 881 882 883 884 885 886 887 888 889 890 891 892
 893 894 895 896 897 898 899 900 901 902 903 904 905 906 907 908 909 910
 911 912 913 914 915 916 917 918 919 920 921 922 923 924 925 926 927 928
 929 930 931 932 933 934 935 936 937 938 939 940 941 942 943 944 945 946
 947 948 949 950 951 952 953 954 955 956 957 958 959 960 961 962 963

/raven/u/cwalker/virtual_environments/yt-env/lib/python3.7/site-packages/ipykernel_launcher.py:109: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.


...

chunk indices: (2579657524,2594149981)
saved
loading chunk 53
chunk indices: (768100274,782592731)
chunk loaded.
Particle list created
chunk loaded.
chunk loaded.
Particle list created
chunk loaded.
chunk loaded.
chunk loaded.
chunk loaded.
Particle list created
Particle list created
Particle list created
Particle list created
Particle list created
Particle list created
offsetfile loaded
offsetfile loadedoffsetfile loadedoffsetfile loaded

offsetfile loaded

offsetfile loaded
offsetfile loaded
offsetfile loaded
ShIDList created
ShIDList created
ShIDList created
ShIDList created
ShIDList created
ShIDList created
ShIDList created
ShIDList created
saved
loading chunk 304
chunk indices: (4405707232,4420199689)
saved
loading chunk 554
chunk indices: (8028821732,8043314189)
saved
loading chunk 679
chunk indices: (9840378982,9854871439)
saved
loading chunk 429
chunk indices: (6217264482,6231756939)
saved
loading chunk 804
chunk indices: (11651936232,11666428689)
saved
loading chunk 54
chunk i

saved
chunk indices: (6318711688,6333204145)loading chunk 311

chunk indices: (4507154438,4521646895)
saved
loading chunk 561
chunk indices: (8130268938,8144761395)
saved
loading chunk 811
chunk indices: (11753383438,11767875895)
saved
loading chunk 686
chunk indices: (9941826188,9956318645)
saved
loading chunk 936
chunk indices: (13564940688,13579433145)
saved
loading chunk 61
chunk indices: (884039938,898532395)
saved
loading chunk 186
chunk indices: (2695597188,2710089645)
chunk loaded.
chunk loaded.
chunk loaded.
chunk loaded.
chunk loaded.
chunk loaded.
chunk loaded.
chunk loaded.
Particle list created
Particle list created
Particle list created
Particle list created
Particle list created
Particle list created
Particle list created
Particle list created
offsetfile loadedoffsetfile loaded
offsetfile loadedoffsetfile loaded
offsetfile loaded


offsetfile loaded
offsetfile loadedoffsetfile loaded

ShIDList created
ShIDList created
ShIDList created
ShIDList created
ShIDList created
Sh

Particle list createdParticle list created

Particle list created
Particle list created
Particle list created
Particle list created
offsetfile loadedoffsetfile loaded
offsetfile loaded
offsetfile loaded

offsetfile loaded
offsetfile loadedoffsetfile loaded
offsetfile loaded

ShIDList created
ShIDList created
ShIDList created
ShIDList created
ShIDList created
ShIDList created
ShIDList created
ShIDList created
saved
loading chunk 444
chunk indices: (6434651352,6449143809)
saved
loading chunk 819
chunk indices: (11869323102,11883815559)
saved
loading chunk 569
chunk indices: (8246208602,8260701059)
saved
loading chunk 944
chunk indices: (13680880352,13695372809)
saved
savedloading chunk 69

loading chunk 694chunk indices: (999979602,1014472059)

chunk indices: (10057765852,10072258309)
chunk loaded.
saved
loading chunk 194
savedchunk indices: (2811536852,2826029309)

loading chunk 319
chunk indices: (4623094102,4637586559)
chunk loaded.
chunk loaded.
Particle list created
chunk loaded.
ch

chunk indices: (11970770308,11985262765)
saved
loading chunk 201
chunk indices: (2912984058,2927476515)
saved
loading chunk 701
chunk indices: (10159213058,10173705515)
saved
loading chunk 451
chunk indices: (6536098558,6550591015)
chunk loaded.
saved
loading chunk 76
chunk indices: (1101426808,1115919265)
chunk loaded.
chunk loaded.
chunk loaded.
Particle list created
chunk loaded.
chunk loaded.
chunk loaded.
chunk loaded.
Particle list created
Particle list created
Particle list created
Particle list created
Particle list created
Particle list created
Particle list created
offsetfile loadedoffsetfile loaded

offsetfile loadedoffsetfile loaded

offsetfile loadedoffsetfile loaded

offsetfile loadedoffsetfile loaded

ShIDList created
ShIDList created
ShIDList created
ShIDList created
ShIDList created
ShIDList created
ShIDList created
ShIDList created
saved
loading chunk 702
chunk indices: (10173705516,10188197973)
saved
loading chunk 577
chunk indices: (8362148266,8376640723)
saved
load

offsetfile loaded
offsetfile loaded
offsetfile loaded
offsetfile loaded
ShIDList createdShIDList created

ShIDList created
ShIDList created
ShIDList created
ShIDList created
ShIDList created
ShIDList created
saved
loading chunk 709
chunk indices: (10275152722,10289645179)
savedsaved

loading chunk 334loading chunk 959

chunk indices: (4840480972,4854973429)chunk indices: (13898267222,13912759679)

saved
loading chunk 584
chunk indices: (8463595472,8478087929)
saved
loading chunk 834
chunk indices: (12086709972,12101202429)
chunk loaded.
saved
loading chunk 459
chunk indices: (6652038222,6666530679)
saved
loading chunk 84
chunk indices: (1217366472,1231858929)
saved
loading chunk 209
chunk indices: (3028923722,3043416179)
chunk loaded.
chunk loaded.
Particle list created
chunk loaded.
chunk loaded.
chunk loaded.
Particle list created
Particle list created
chunk loaded.
chunk loaded.
Particle list created
Particle list created
Particle list created
Particle list created
Particle list cre